In [ ]:
!wget https://storage.googleapis.com/quickdraw_dataset/sketchrnn/cat.npz

In [ ]:
!pip install einops

In [ ]:
!pip install cairosvg

In [ ]:
!pip install svgwrite

In [ ]:
import math
from typing import Optional, Tuple, Any

import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from torch import optim
from torch.utils.data import Dataset, DataLoader

import einops

from PIL import Image
import cairosvg
import os
from six.moves import range
import svgwrite
import io
import torch.nn.functional as F
from datetime import datetime
from typing import Optional, Tuple, Any, List


In [ ]:
def get_bounds(data, factor=10):
    """Return bounds of data."""
    min_x = 0
    max_x = 0
    min_y = 0
    max_y = 0

    abs_x = 0
    abs_y = 0
    for i in range(len(data)):
        x = float(data[i, 0]) / factor
        y = float(data[i, 1]) / factor
        abs_x += x
        abs_y += y
        min_x = min(min_x, abs_x)
        min_y = min(min_y, abs_y)
        max_x = max(max_x, abs_x)
        max_y = max(max_y, abs_y)

    return min_x, max_x, min_y, max_y

In [ ]:
def draw_strokes(data, svg_filename, factor=0.2, padding=50):
    """
    little function that displays vector images and saves them to .svg
    :param data:
    :param factor:
    :param svg_filename:
    :param padding:
    :return:
    """
    min_x, max_x, min_y, max_y = get_bounds(data, factor)
    dims = (padding + max_x - min_x, padding + max_y - min_y)
    dwg = svgwrite.Drawing(size=dims)
    dwg.add(dwg.rect(insert=(0, 0), size=dims, fill='white'))
    lift_pen = 1
    abs_x = int(padding / 2) - min_x
    abs_y = int(padding / 2) - min_y
    p = "M%s, %s " % (abs_x, abs_y)
    # use lowcase for relative position
    command = "m"

    for i in range(len(data)):
        if lift_pen == 1:
            command = "m"
        elif command != "l":
            command = "l"
        else:
            command = ""
        x = float(data[i, 0]) / factor
        y = float(data[i, 1]) / factor
        lift_pen = data[i, 2]
        p += command + str(x) + ", " + str(y) + " "
    the_color = "black"
    stroke_width = 1

    dwg.add(dwg.path(p).stroke(the_color, stroke_width).fill("none"))
    #dwg.save()

    svg_code = dwg.tostring()
    img = cairosvg.svg2png(bytestring=svg_code)
    image = Image.open(io.BytesIO(img))
    image = image.resize((48,48))
    #image = image.convert('1')
    aarr = np.asarray(image)
    aarr = np.where(aarr < 255, 0, 255)
    aarr = aarr[:,:,1]
    # aarr = np.reshape(aarr, (28*28))
    # np.save('array', aarr)
    #image.save(svg_filename + '.png')
    return dims, dwg.tostring(), aarr

In [ ]:
def load_dataset(data_dir, data_set, percentage):
    """Loads the .npz file, and splits the set into train/valid/test."""

    img_H, img_W = 48, 48

    if isinstance(data_set, list):
        datasets = data_set
    else:
        datasets = [data_set]

    train_strokes = None
    valid_strokes = None
    test_strokes = None
    
    nb_strokes = {}
    png_paths_map = {'train': [], 'valid': [], 'test': []}

    for dataset in datasets:
        
        data_filepath = os.path.join(data_dir, 'data', dataset)
        data = np.load(data_filepath, encoding='latin1', allow_pickle=True)
        print('Loaded {}/{}/{} from {}'.format(
            len(data['train'][:int(len(data['train']) * percentage)]),
            len(data['valid'][:int(len(data['valid']) * percentage)] ),
            len(data['test'][:int(len(data['test']) * percentage)] ),
            dataset))
        
        nb_strokes[dataset] = { 'train':int(len(data['train']) * percentage),
                                'valid':int(len(data['valid']) * percentage),
                                'test':int(len(data['test']) * percentage) }

        
        if train_strokes is None:
            train_strokes = data['train'][:int(len(data['train']) * percentage)]  # [N (#sketches),], each with [S (#points), 3]
            valid_strokes = data['valid'][:int(len(data['valid']) * percentage)] 
            test_strokes = data['test'][:int(len(data['test']) * percentage)] 
        else:
            train_strokes = np.concatenate((train_strokes, data['train'][:int(len(data['train']) * percentage)] ))
            valid_strokes = np.concatenate((valid_strokes, data['valid'][:int(len(data['valid']) * percentage)] ))
            test_strokes = np.concatenate((test_strokes, data['test'][:int(len(data['test']) * percentage)] ))

        splits = ['train', 'valid', 'test']
        for split in splits:
            for im_idx in range(len(data[split][:int(len(data[split]) * percentage)] ) ):
                png_path = os.path.join(data_dir, 'png', dataset[:-4], split,
                                        str(img_H) + 'x' + str(img_W), str(im_idx) + '.png')
                png_paths_map[split].append(png_path)

    all_strokes = np.concatenate((train_strokes, valid_strokes, test_strokes))
    num_points = 0
    for stroke in all_strokes:
        num_points += len(stroke)
    avg_len = num_points / len(all_strokes)
    print('Dataset combined: {} ({}/{}/{}), avg len {}'.format(
        len(all_strokes), len(train_strokes), len(valid_strokes),
        len(test_strokes), int(avg_len)))
    assert len(train_strokes) == len(png_paths_map['train'])
    assert len(valid_strokes) == len(png_paths_map['valid'])
    assert len(test_strokes) == len(png_paths_map['test'])

    result = {'train':train_strokes, 'valid': valid_strokes, 'test': test_strokes}
    png_paths = {'train':png_paths_map['train'], 'valid':png_paths_map['valid'], 'test':png_paths_map['test']}
    return result, png_paths, nb_strokes

In [ ]:
def pad_image(png_filename, pngsize):
    curr_png = Image.open(png_filename).convert('RGB')
    png_curr_w = curr_png.width
    png_curr_h = curr_png.height
    print("pngsize: {}, {}".format(pngsize[0],pngsize[1]))

    if png_curr_w != pngsize[0] and png_curr_h != pngsize[1]:
        print('Not aligned', 'png_curr_w', png_curr_w, 'png_curr_h', png_curr_h)

    padded_png = np.zeros(shape=[pngsize[1], pngsize[0], 3], dtype=np.uint8)
    padded_png.fill(255)

    if png_curr_w > png_curr_h:
        pad = int(round((png_curr_w - png_curr_h) / 2))
        padded_png[pad: pad + png_curr_h, :png_curr_w, :] = np.array(curr_png, dtype=np.uint8)
    else:
        pad = int(round((png_curr_h - png_curr_w) / 2))
        padded_png[:png_curr_h, pad: pad + png_curr_w, :] = np.array(curr_png, dtype=np.uint8)

    padded_png = Image.fromarray(padded_png, 'RGB')
    padded_png.save(png_filename, 'PNG')

In [ ]:
def svg2png(dwg_string, svgsize, pngsize, png_filename, padding=False):
    """convert svg into png, using cairosvg"""
    svg_w, svg_h = svgsize
    png_w, png_h = pngsize
    x_scale = png_w / svg_w
    y_scale = png_h / svg_h

    if x_scale > y_scale:
        cairosvg.svg2png(bytestring=dwg_string, write_to=png_filename, output_height=png_h)
    else:
        cairosvg.svg2png(bytestring=dwg_string, write_to=png_filename, output_width=png_w)

    if padding:
        pad_image(png_filename, pngsize)
    img = Image.open(png_filename)
    img = img.resize((48,48))
    img = np.asarray(img)
    img = np.where(img < 254, 0, 255).astype(np.uint8)
    #plt.imshow(img)
    img = Image.fromarray(img)
    img.save(png_filename, 'PNG')

In [ ]:
def render_svg2bitmap(data_base_dir, data_set, percentage):
    """get sketch as png"""
    img_H, img_W = 250, 250

    npz_dir = os.path.join(data_base_dir, 'npz')
    svg_dir = os.path.join(data_base_dir, 'svg')
    png_dir = os.path.join(data_base_dir, 'png')

    datasets, png_paths, nb_strokes = load_dataset(data_base_dir, data_set, percentage)
    
    nb_img = {'train':0, 'valid':0, 'test':0}
    
    for dataset_i in data_set:
        assert dataset_i[-4:] == '.npz'
        cate_svg_dir = os.path.join(svg_dir, dataset_i[:-4])
        cate_png_dir = os.path.join(png_dir, dataset_i[:-4])

        data_types = ['train', 'valid', 'test']
        for data_type in data_types:
            split_cate_svg_dir = os.path.join(cate_svg_dir, data_type)
            split_cate_png_dir = os.path.join(cate_png_dir, data_type,
                                              str(48) + 'x' + str(48))

            os.makedirs(split_cate_svg_dir, exist_ok=True)
            os.makedirs(split_cate_png_dir, exist_ok=True)

            split_dataset = datasets[data_type]
            split_png_path = png_paths[data_type]
            nb_strokes_dataset = nb_strokes[dataset_i][data_type]             

            for _ in range(nb_strokes_dataset):                
                stroke = np.copy(split_dataset[nb_img[data_type]])                
                png_path = split_png_path[nb_img[data_type]]
                print(png_path, 'stroke.shape', stroke.shape)
                actual_idx = png_path[len(split_cate_png_dir) + 1:-4]
                svg_path = os.path.join(split_cate_svg_dir, str(actual_idx) + '.svg')
                
        
                svg_size, dwg_bytestring, _ = draw_strokes(stroke, svg_path, padding=10)  # (w, h)
                svg2png(dwg_bytestring, svg_size, (img_W, img_H), png_path,
                               padding=True)
                nb_img[data_type] += 1
                
#render_svg2bitmap('./', ['duck.npz', 'elephant.npz', 'giraffe.npz'], 0.001)

In [ ]:
"""
import matplotlib.pyplot as plt
datasets, png_paths, _ = load_dataset('./', ['cat.npz'], 0.01)
stroke = np.copy(datasets['train'][0])
svg_size, dwg_bytestring, aarr = draw_strokes(stroke, './', padding=10)
#cairosvg.svg2png(bytestring=dwg_bytestring, write_to='./cat.png')

#svg2png(dwg_bytestring, svg_size, (250, 250), './cat.png',padding=True)
#print(png_paths['train'][0])
#print(aarr.shape)
#plt.imshow(aarr)
"""

In [ ]:
class StrokesDataset(Dataset):
    """
    ## Dataset
    This class loads and pre-processes the data.
    """

    def __init__(self, dataset: np.array, max_seq_length: int, png_paths:list, scale: Optional[float] = None):
        """
        `dataset` is a list of numpy arrays of shape [seq_len, 3].
        It is a sequence of strokes, and each stroke is represented by
        3 integers.
        First two are the displacements along x and y ($\Delta x$, $\Delta y$)
        and the last integer represents the state of the pen, $1$ if it's touching
        the paper and $0$ otherwise.
        """
        data = []
        # We iterate through each of the sequences and filter
        for seq in dataset:
            # Filter if the length of the sequence of strokes is within our range
            if 10 < len(seq) <= max_seq_length:
                # Clamp $\Delta x$, $\Delta y$ to $[-1000, 1000]$
                seq = np.minimum(seq, 1000)
                seq = np.maximum(seq, -1000)
                # Convert to a floating point array and add to `data`
                seq = np.array(seq, dtype=np.float32)
                data.append(seq)

        # We then calculate the scaling factor which is the
        # standard deviation of ($\Delta x$, $\Delta y$) combined.
        # Paper notes that the mean is not adjusted for simplicity,
        # since the mean is anyway close to $0$.
        if scale is None:
            scale = np.std(np.concatenate([np.ravel(s[:, 0:2]) for s in data]))
        self.scale = scale

        # Get the longest sequence length among all sequences
        longest_seq_len = max([len(seq) for seq in data])

        # We initialize PyTorch data array with two extra steps for start-of-sequence (sos)
        # and end-of-sequence (eos).
        # Each step is a vector $(\Delta x, \Delta y, p_1, p_2, p_3)$.
        # Only one of $p_1, p_2, p_3$ is $1$ and the others are $0$.
        # They represent *pen down*, *pen up* and *end-of-sequence* in that order.
        # $p_1$ is $1$ if the pen touches the paper in the next step.
        # $p_2$ is $1$ if the pen doesn't touch the paper in the next step.
        # $p_3$ is $1$ if it is the end of the drawing.
        self.data = torch.zeros(len(data), longest_seq_len + 2, 5, dtype=torch.float)
        # The mask array needs only one extra-step since it is for the outputs of the
        # decoder, which takes in `data[:-1]` and predicts next step.
        self.mask = torch.zeros(len(data), longest_seq_len + 1)

        for i, seq in enumerate(data):
            seq = torch.from_numpy(seq)
            len_seq = len(seq)
            # Scale and set $\Delta x, \Delta y$
            self.data[i, 1:len_seq + 1, :2] = seq[:, :2] / scale
            # $p_1$
            self.data[i, 1:len_seq + 1, 2] = 1 - seq[:, 2]
            # $p_2$
            self.data[i, 1:len_seq + 1, 3] = seq[:, 2]
            # $p_3$
            self.data[i, len_seq + 1:, 4] = 1
            # Mask is on until end of sequence
            self.mask[i, :len_seq + 1] = 1

        # Start-of-sequence is $(0, 0, 1, 0, 0)$
        self.data[:, 0, 2] = 1

        self.png_paths = png_paths

    def __len__(self):
        """Size of the dataset"""
        return len(self.data)

    def __getitem__(self, idx: int):
        """Get a sample"""
        img = Image.open(self.png_paths[idx]).convert('L')
        img = np.array(img)
        return self.data[idx], self.mask[idx], img

In [ ]:
"""
data_sets = ['cat.npz', 'duck.npz']
datasets, png_paths, _ = load_dataset('./', data_sets, 0.01)
max_seq_length = 150
# Create training dataset
train_dataset = StrokesDataset(datasets['train'], max_seq_length, png_paths['train'])
_,_, img = train_dataset[0]
print(img.shape)
plt.imshow(img)
"""

In [ ]:
class BivariateGaussianMixture:
    """
    ## Bi-variate Gaussian mixture
    The mixture is represented by $\Pi$ and
    $\mathcal{N}(\mu_{x}, \mu_{y}, \sigma_{x}, \sigma_{y}, \rho_{xy})$.
    This class adjusts temperatures and creates the categorical and Gaussian
    distributions from the parameters.
    """

    def __init__(self, pi_logits: torch.Tensor, mu_x: torch.Tensor, mu_y: torch.Tensor,
                 sigma_x: torch.Tensor, sigma_y: torch.Tensor, rho_xy: torch.Tensor):
        self.pi_logits = pi_logits
        self.mu_x = mu_x
        self.mu_y = mu_y
        self.sigma_x = sigma_x
        self.sigma_y = sigma_y
        self.rho_xy = rho_xy

    @property
    def n_distributions(self):
        """Number of distributions in the mixture, $M$"""
        return self.pi_logits.shape[-1]

    def set_temperature(self, temperature: float):
        """
        Adjust by temperature $\tau$
        """
        # $$\hat{\Pi_k} \leftarrow \frac{\hat{\Pi_k}}{\tau}$$
        self.pi_logits /= temperature
        # $$\sigma^2_x \leftarrow \sigma^2_x \tau$$
        self.sigma_x *= math.sqrt(temperature)
        # $$\sigma^2_y \leftarrow \sigma^2_y \tau$$
        self.sigma_y *= math.sqrt(temperature)

    def get_distribution(self):
        # Clamp $\sigma_x$, $\sigma_y$ and $\rho_{xy}$ to avoid getting `NaN`s
        sigma_x = torch.clamp_min(self.sigma_x, 1e-5)
        sigma_y = torch.clamp_min(self.sigma_y, 1e-5)
        rho_xy = torch.clamp(self.rho_xy, -1 + 1e-5, 1 - 1e-5)

        # Get means
        mean = torch.stack([self.mu_x, self.mu_y], -1)
        # Get covariance matrix
        cov = torch.stack([
            sigma_x * sigma_x, rho_xy * sigma_x * sigma_y,
            rho_xy * sigma_x * sigma_y, sigma_y * sigma_y
        ], -1)
        cov = cov.view(*sigma_y.shape, 2, 2)

        # Create bi-variate normal distribution.
        #
        # 📝 It would be efficient to `scale_tril` matrix as `[[a, 0], [b, c]]`
        # where
        # $$a = \sigma_x, b = \rho_{xy} \sigma_y, c = \sigma_y \sqrt{1 - \rho^2_{xy}}$$.
        # But for simplicity we use co-variance matrix.
        # [This is a good resource](https://www2.stat.duke.edu/courses/Spring12/sta104.1/Lectures/Lec22.pdf)
        # if you want to read up more about bi-variate distributions, their co-variance matrix,
        # and probability density function.
        multi_dist = torch.distributions.MultivariateNormal(mean, covariance_matrix=cov)

        # Create categorical distribution $\Pi$ from logits
        cat_dist = torch.distributions.Categorical(logits=self.pi_logits)

        #
        return cat_dist, multi_dist

In [ ]:
class EncoderCNN(nn.Module):
    """
    ## Encoder module
    This consists of a bidirectional LSTM
    """

    def __init__(self, d_z: int):
        super().__init__()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.filter_hp = torch.tensor([[[[-1, -1, -1],
                                             [-1, 8, -1],
                                             [-1, -1, -1]]]]).float().to(self.device)

        self.conv1 = nn.Conv2d(in_channels = 1, out_channels= 4, kernel_size = 2, stride =2)
        self.conv2 = nn.Conv2d(in_channels = 4, out_channels= 4, kernel_size = 2, stride =1)
        self.conv3 = nn.Conv2d(in_channels = 4, out_channels= 8, kernel_size = 2, stride =2)
        self.conv4 = nn.Conv2d(in_channels = 8, out_channels= 8, kernel_size = 2, stride =1)
        self.conv5 = nn.Conv2d(in_channels = 8, out_channels= 8, kernel_size = 2, stride =2)
        self.conv6 = nn.Conv2d(in_channels = 8, out_channels= 8, kernel_size = 2, stride =1)

        # Head to get $\mu$
        self.mu_head = nn.Linear(128, d_z)
        # Head to get $\hat{\sigma}$
        self.sigma_head = nn.Linear(128, d_z)

    def high_pass_filtering(self, img_in):
        """
        high pass filtering
        :param img_in: [N, H, W, 1]
        :return: img_out: [N, H, W, 1]
        """
        img_out = F.conv2d(img_in, self.filter_hp)
        return img_out


    def forward(self, inputs: torch.Tensor, state=None):
        x = self.high_pass_filtering(inputs)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = torch.tanh(self.conv6(x))
        
        x = x.view(inputs.shape[0], -1)

        # $\mu$
        mu = self.mu_head(x)
        # $\hat{\sigma}$
        sigma_hat = self.sigma_head(x)
        # $\sigma = \exp(\frac{\hat{\sigma}}{2})$
        sigma = torch.exp(sigma_hat / 2.)

        # Sample $z = \mu + \sigma \cdot \mathcal{N}(0, I)$
        z = mu + sigma * torch.normal(mu.new_zeros(mu.shape), mu.new_ones(mu.shape))
      
        return z, mu, sigma_hat

In [ ]:
"""
test_array = torch.rand((2,1 ,48,48))
encoder = EncoderRNN(128)
encoder(test_array)
"""

In [ ]:
class DecoderRNN(nn.Module):
    """
    ## Decoder module
    This consists of a LSTM
    """

    def __init__(self, d_z: int, dec_hidden_size: int, n_distributions: int):
        super().__init__()
        # LSTM takes $[(\Delta x, \Delta y, p_1, p_2, p_3); z]$ as input
        self.lstm = nn.LSTM(d_z + 5, dec_hidden_size)

        # Initial state of the LSTM is $[h_0; c_0] = \tanh(W_{z}z + b_z)$.
        # `init_state` is the linear transformation for this
        self.init_state = nn.Linear(d_z, 2 * dec_hidden_size)

        # This layer produces outputs for each of the `n_distributions`.
        # Each distribution needs six parameters
        # $(\hat{\Pi_i}, \mu_{x_i}, \mu_{y_i}, \hat{\sigma_{x_i}}, \hat{\sigma_{y_i}} \hat{\rho_{xy_i}})$
        self.mixtures = nn.Linear(dec_hidden_size, 6 * n_distributions)

        # This head is for the logits $(\hat{q_1}, \hat{q_2}, \hat{q_3})$
        self.q_head = nn.Linear(dec_hidden_size, 3)
        # This is to calculate $\log(q_k)$ where
        # $$q_k = \operatorname{softmax}(\hat{q})_k = \frac{\exp(\hat{q_k})}{\sum_{j = 1}^3 \exp(\hat{q_j})}$$
        self.q_log_softmax = nn.LogSoftmax(-1)

        # These parameters are stored for future reference
        self.n_distributions = n_distributions
        self.dec_hidden_size = dec_hidden_size

    def forward(self, x: torch.Tensor, z: torch.Tensor, state: Optional[Tuple[torch.Tensor, torch.Tensor]]):
        # The target/expected vectors of strokes
        #self.output_x = self.input_data[:, 1:self.hps.max_seq_len + 1, :]  # [N, max_seq_len, 5]
        # vectors of strokes to be fed to decoder (same as above, but lagged behind
        # one step to include initial dummy value of (0, 0, 1, 0, 0))
        #self.input_x = self.input_data[:, :self.hps.max_seq_len, :]  # [N, max_seq_len, 5]

        # Calculate the initial state
        if state is None:
            # $[h_0; c_0] = \tanh(W_{z}z + b_z)$
            h, c = torch.split(torch.tanh(self.init_state(z)), self.dec_hidden_size, 1)
            # `h` and `c` have shapes `[batch_size, lstm_size]`. We want to shape them
            # to `[1, batch_size, lstm_size]` because that's the shape used in LSTM.
            state = (h.unsqueeze(0).contiguous(), c.unsqueeze(0).contiguous())
            #state = torch.cat(state, stroke)

        # Run the LSTM
        outputs, state = self.lstm(x, state)

        # Get $\log(q)$
        q_logits = self.q_log_softmax(self.q_head(outputs))

        # Get $(\hat{\Pi_i}, \mu_{x,i}, \mu_{y,i}, \hat{\sigma_{x,i}},
        # \hat{\sigma_{y,i}} \hat{\rho_{xy,i}})$.
        # `torch.split` splits the output into 6 tensors of size `self.n_distribution`
        # across dimension `2`.
        pi_logits, mu_x, mu_y, sigma_x, sigma_y, rho_xy = \
            torch.split(self.mixtures(outputs), self.n_distributions, 2)

        # Create a bi-variate Gaussian mixture
        # $\Pi$ and 
        # $\mathcal{N}(\mu_{x}, \mu_{y}, \sigma_{x}, \sigma_{y}, \rho_{xy})$
        # where
        # $$\sigma_{x,i} = \exp(\hat{\sigma_{x,i}}), \sigma_{y,i} = \exp(\hat{\sigma_{y,i}}),
        # \rho_{xy,i} = \tanh(\hat{\rho_{xy,i}})$$
        # and
        # $$\Pi_i = \operatorname{softmax}(\hat{\Pi})_i = \frac{\exp(\hat{\Pi_i})}{\sum_{j = 1}^3 \exp(\hat{\Pi_j})}$$
        #
        # $\Pi$ is the categorical probabilities of choosing the distribution out of the mixture
        # $\mathcal{N}(\mu_{x}, \mu_{y}, \sigma_{x}, \sigma_{y}, \rho_{xy})$.
        dist = BivariateGaussianMixture(pi_logits, mu_x, mu_y,
                                        torch.exp(sigma_x), torch.exp(sigma_y), torch.tanh(rho_xy))

        #
        return dist, q_logits, state

In [ ]:
"""
batchSize= 3
test_array = torch.rand((batchSize,1 ,48,48))
encoder = EncoderRNN(128)
z, mu, sigma_hat = encoder(test_array)
print(z.shape)
# Decode the mixture of distributions and $\hat{q}$
strokes_array = torch.rand(96, batchSize, 5)
print(strokes_array.shape)
# Concatenate $[(\Delta x, \Delta y, p_1, p_2, p_3); z]$
z_stack = z.unsqueeze(0).expand(strokes_array.shape[0] - 1, -1, -1)
print(z_stack.shape)
inputs = torch.cat([strokes_array[:-1], z_stack], 2)

decoder = DecoderRNN(128, 512, 20)
dist, q_logits, _ = decoder(inputs, z, 0, None, None)
print(dist)
"""

In [ ]:
class ReconstructionLoss(nn.Module):
    """
    ## Reconstruction Loss
    """

    def forward(self, mask: torch.Tensor, target: torch.Tensor,
                 dist: 'BivariateGaussianMixture', q_logits: torch.Tensor):
        # Get $\Pi$ and $\mathcal{N}(\mu_{x}, \mu_{y}, \sigma_{x}, \sigma_{y}, \rho_{xy})$
        pi, mix = dist.get_distribution()
        # `target` has shape `[seq_len, batch_size, 5]` where the last dimension is the features
        # $(\Delta x, \Delta y, p_1, p_2, p_3)$.
        # We want to get $\Delta x, \Delta$ y and get the probabilities from each of the distributions
        # in the mixture $\mathcal{N}(\mu_{x}, \mu_{y}, \sigma_{x}, \sigma_{y}, \rho_{xy})$.
        #
        # `xy` will have shape `[seq_len, batch_size, n_distributions, 2]`
        xy = target[:, :, 0:2].unsqueeze(-2).expand(-1, -1, dist.n_distributions, -1)
        # Calculate the probabilities
        # $$p(\Delta x, \Delta y) =
        # \sum_{j=1}^M \Pi_j \mathcal{N} \big( \Delta x, \Delta y \vert
        # \mu_{x,j}, \mu_{y,j}, \sigma_{x,j}, \sigma_{y,j}, \rho_{xy,j}
        # \big)$$
        probs = torch.sum(pi.probs * torch.exp(mix.log_prob(xy)), 2)

        # $$L_s = - \frac{1}{N_{max}} \sum_{i=1}^{N_s} \log \big (p(\Delta x, \Delta y) \big)$$
        # Although `probs` has $N_{max}$ (`longest_seq_len`) elements, the sum is only taken
        # upto $N_s$ because the rest is masked out.
        #
        # It might feel like we should be taking the sum and dividing by $N_s$ and not $N_{max}$,
        # but this will give higher weight for individual predictions in shorter sequences.
        # We give equal weight to each prediction $p(\Delta x, \Delta y)$ when we divide by $N_{max}$
        loss_stroke = -torch.mean(mask * torch.log(1e-5 + probs))

        # $$L_p = - \frac{1}{N_{max}} \sum_{i=1}^{N_{max}} \sum_{k=1}^{3} p_{k,i} \log(q_{k,i})$$
        loss_pen = -torch.mean(target[:, :, 2:] * q_logits)

        # $$L_R = L_s + L_p$$
        return loss_stroke + loss_pen


In [ ]:
class Sampler:
    """
    ## Sampler
    This samples a sketch from the decoder and plots it
    """

    def __init__(self, encoder: EncoderCNN, decoder: DecoderRNN):
        self.decoder = decoder
        self.encoder = encoder
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def sample(self, data: torch.Tensor, temperature: float, filename: Optional[str] = None):
        # $N_{max}$
        longest_seq_len = len(data[0])
        
        # Get $z$ from the encoder
        img = torch.tensor(data[2]).unsqueeze(0).unsqueeze(0).float().to(self.device)        
        data = data[0]
        z, _, _ = self.encoder(img)

        # Start-of-sequence stroke is $(0, 0, 1, 0, 0)$
        s = data.new_tensor([0, 0, 1, 0, 0]).to(self.device)
        seq = [s]
        # Initial decoder is `None`.
        # The decoder will initialize it to $[h_0; c_0] = \tanh(W_{z}z + b_z)$
        state = None

        # We don't need gradients
        with torch.no_grad():
            # Sample $N_{max}$ strokes
            for i in range(longest_seq_len):
                # $[(\Delta x, \Delta y, p_1, p_2, p_3); z]$ is the input to the decoder
                data = torch.cat([s.view(1, 1, -1), z.unsqueeze(0)], 2)
                # Get $\Pi$, $\mathcal{N}(\mu_{x}, \mu_{y}, \sigma_{x}, \sigma_{y}, \rho_{xy})$,
                # $q$ and the next state from the decoder
                dist, q_logits, state = self.decoder(data, z, state)
                # Sample a stroke
                s = self._sample_step(dist, q_logits, temperature)
                # Add the new stroke to the sequence of strokes
                seq.append(s)
                # Stop sampling if $p_3 = 1$. This indicates that sketching has stopped
                if s[4] == 1:
                    break

        # Create a PyTorch tensor of the sequence of strokes
        seq = torch.stack(seq)

        # Plot the sequence of strokes
        self.plot(seq, filename)

    @staticmethod
    def _sample_step(dist: 'BivariateGaussianMixture', q_logits: torch.Tensor, temperature: float):
        # Set temperature $\tau$ for sampling. This is implemented in class `BivariateGaussianMixture`.
        dist.set_temperature(temperature)
        # Get temperature adjusted $\Pi$ and $\mathcal{N}(\mu_{x}, \mu_{y}, \sigma_{x}, \sigma_{y}, \rho_{xy})$
        pi, mix = dist.get_distribution()
        # Sample from $\Pi$ the index of the distribution to use from the mixture
        idx = pi.sample()[0, 0]

        # Create categorical distribution $q$ with log-probabilities `q_logits` or $\hat{q}$
        q = torch.distributions.Categorical(logits=q_logits / temperature)
        # Sample from $q$
        q_idx = q.sample()[0, 0]

        # Sample from the normal distributions in the mixture and pick the one indexed by `idx`
        xy = mix.sample()[0, 0, idx]

        # Create an empty stroke $(\Delta x, \Delta y, q_1, q_2, q_3)$
        stroke = q_logits.new_zeros(5)
        # Set $\Delta x, \Delta y$
        stroke[:2] = xy
        # Set $q_1, q_2, q_3$
        stroke[q_idx + 2] = 1
        #
        return stroke

    @staticmethod
    def plot(seq: torch.Tensor, filename: Optional[str] = None):
        # Take the cumulative sums of $(\Delta x, \Delta y)$ to get $(x, y)$
        seq[:, 0:2] = torch.cumsum(seq[:, 0:2], dim=0)
        # Create a new numpy array of the form $(x, y, q_2)$
        seq[:, 2] = seq[:, 3]
        seq = seq[:, 0:3].detach().cpu().numpy()

        # Split the array at points where $q_2$ is $1$.
        # i.e. split the array of strokes at the points where the pen is lifted from the paper.
        # This gives a list of sequence of strokes.
        strokes = np.split(seq, np.where(seq[:, 2] > 0)[0] + 1)
        # Plot each sequence of strokes
        for s in strokes:
            plt.plot(s[:, 0], -s[:, 1])
        # Don't show axes
        plt.axis('off')
        # Show the plot
        if filename:
            plt.savefig(filename)
        else:
            plt.show()

In [ ]:
class Configs():
    """
    ## Configurations
    These are default configurations which can later be adjusted by passing a `dict`.
    """

    # Device configurations to pick the device to run the experiment
    #device: torch.device = DeviceConfigs()
    #
    encoder: EncoderCNN
    decoder: DecoderRNN
    optimizer: optim.Adam
    sampler: Sampler

    dataset_name: str
    train_loader: DataLoader
    valid_loader: DataLoader
    train_dataset: StrokesDataset
    valid_dataset: StrokesDataset

    # Encoder and decoder sizes
    enc_hidden_size = 256
    dec_hidden_size = 512

    # Batch size
    batch_size = 100

    # Number of features in $z$
    d_z = 128
    # Number of distributions in the mixture, $M$
    n_distributions = 20

    # Weight of KL divergence loss, $w_{KL}$
    #kl_div_loss_weight = 0.5
    # Gradient clipping
    grad_clip = 1.
    # Temperature $\tau$ for sampling
    temperature = 0.4

    # Filter out stroke sequences longer than $200$
    max_seq_length = 200

    epochs = 100

    #kl_div_loss = KLDivLoss()
    reconstruction_loss = ReconstructionLoss()

    def __init__(self, classes: List[str], device_id: int):
        self.last_loss = None
        self.device = f'cuda:{device_id}' if torch.cuda.is_available() else 'cpu'
        # Initialize encoder & decoder
        self.encoder = EncoderCNN(self.d_z).to(self.device)
        self.decoder = DecoderRNN(self.d_z, self.dec_hidden_size, self.n_distributions).to(self.device)

        # Set optimizer. Things like type of optimizer and learning rate are configurable
        #optimizer = OptimizerConfigs()
        #optimizer.parameters = list(self.encoder.parameters()) + list(self.decoder.parameters())
        #self.optimizer = optimizer
        self.optimizer = optim.Adam(list(self.encoder.parameters()) + list(self.decoder.parameters()), lr=0.0001)

        # Create sampler
        self.sampler = Sampler(self.encoder, self.decoder)

        # `npz` file path is `data/sketch/[DATASET NAME].npz`
        #data_sets = ['cat.npz', 'duck.npz', 'owl.npz', 'giraffe.npz']
        data_sets = [f'{c}.npz' for c in classes]
        percentage = 0.1
        datasets, png_paths, _ = load_dataset('./', data_sets, percentage)
        if not os.path.exists(png_paths['test'][-1]): 
            render_svg2bitmap('./', data_sets, percentage)

        # Create training dataset
        self.train_dataset = StrokesDataset(datasets['train'], self.max_seq_length, png_paths['train'])
        # Create validation dataset
        self.valid_dataset = StrokesDataset(datasets['valid'], self.max_seq_length, png_paths['valid'], self.train_dataset.scale)

        # Create training data loader
        self.train_loader = DataLoader(self.train_dataset, self.batch_size, shuffle=True)
        # Create validation data loader
        self.valid_loader = DataLoader(self.valid_dataset, self.batch_size)

        self.state_modules = []

        self.sampling_images_indices = None
        self.train_images_indices = None

    def save(self, n_images=12):
        
        # Init at first time
        if not self.sampling_images_indices:
            self.sampling_images_indices = [np.random.choice(len(self.valid_dataset)) for i in range(n_images)]
        if not self.train_images_indices:
            self.train_images_indices = [np.random.choice(len(self.train_dataset)) for i in range(n_images)]

        dirname = 'checkpoints'
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        now = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')

        torch.save({
            'encoder': self.encoder.state_dict(),
            'decoder': self.decoder.state_dict()
        }, os.path.join(dirname, f'model-{now}.dict'))

        plt.close()
        plt.figure(figsize=(28,20))
        plt.tight_layout()

        plt.subplot(n_images, 6, 1)
        filename = os.path.join(dirname, f'model-{now}-train.png')
        for i, index in enumerate(self.sampling_images_indices):
            plt.subplot(n_images, 6, i*6+1)
            plt.title(f'Train dataset image {index}')
            data_ = self.train_dataset[index]
            Sampler.plot(data_[0].clone(), filename)
            #data_ = data.unsqueeze(1).to(self.device)
            for j, temperature in enumerate(np.linspace(0.01, 0.99, 5)):
                plt.subplot(n_images, 6, i*6+2+j)
                plt.title(f'temperature={temperature}')
                self.sampler.sample(data_, temperature, filename)

        plt.close()
        plt.figure(figsize=(20,18))
        plt.tight_layout()

        plt.subplot(n_images, 6, 1)
        filename = os.path.join(dirname, f'model-{now}-validation.png')
        for i, index in enumerate(self.sampling_images_indices):
            plt.subplot(n_images, 6, i*6+1)
            plt.title(f'Validation dataset image {index}')
            data_ = self.valid_dataset[index]
            Sampler.plot(data_[0].clone(), filename)
            #data_ = data.unsqueeze(1).to(self.device)
            for j, temperature in enumerate(np.linspace(0.01, 0.99, 5)):
                plt.subplot(n_images, 6, i*6+2+j)
                plt.title(f'temperature={temperature}')
                self.sampler.sample(data_, temperature, filename)

                
    def load(self, path):
        checkpoint = torch.load(path)
        self.encoder.load_state_dict(checkpoint['encoder'])
        self.decoder.load_state_dict(checkpoint['decoder'])


    def step(self, batch: Any, is_train=True):
        self.encoder.train(is_train)
        self.decoder.train(is_train)

        # Move `data` and `mask` to device and swap the sequence and batch dimensions.
        # `data` will have shape `[seq_len, batch_size, 5]` and
        # `mask` will have shape `[seq_len, batch_size]`.
        data = batch[0].to(self.device).transpose(0, 1)
        mask = batch[1].to(self.device).transpose(0, 1)
        img = batch[2].to(self.device).view(len(batch[2]), 1, 48,48).float()

        z, mu, sigma_hat = self.encoder(img)

        # Decode the mixture of distributions and $\hat{q}$

        # Concatenate $[(\Delta x, \Delta y, p_1, p_2, p_3); z]$
        z_stack = z.unsqueeze(0).expand(data.shape[0] - 1, -1, -1)
        inputs = torch.cat([data[:-1], z_stack], 2)
        # Get mixture of distributions and $\hat{q}$
        dist, q_logits, _ = self.decoder(inputs, z, None)

        # Compute the loss
        # $L_{KL}$
        #kl_loss = self.kl_div_loss(sigma_hat, mu)
        # $L_R$
        reconstruction_loss = self.reconstruction_loss(mask, data[1:], dist, q_logits)

        # $Loss = L_R + w_{KL} L_{KL}$
        loss = reconstruction_loss # + self.kl_div_loss_weight * kl_loss

        self.last_loss = loss

        # Only if we are in training state
        if is_train:

            # Set `grad` to zero
            self.optimizer.zero_grad()            
            # Compute gradients
            loss.backward()
            #print(loss.item(), end='\r')
            # Clip gradients
            nn.utils.clip_grad_norm_(self.encoder.parameters(), self.grad_clip)
            nn.utils.clip_grad_norm_(self.decoder.parameters(), self.grad_clip)
            # Optimize
            self.optimizer.step()
        


    def sample(self, filename: Optional[str] = None):
        # Randomly pick a sample from validation dataset to encoder
        data = self.valid_dataset[np.random.choice(len(self.valid_dataset))]
        
        # Add batch dimension and move it to device
        #data = data.unsqueeze(1).to(self.device)
        # Sample
        self.sampler.sample(data, self.temperature, filename)

In [ ]:
#classes = ['cat', 'pig']
classes = ['cat']
device_id = 0
c = Configs(classes, device_id)

In [ ]:
from tqdm import tqdm
epochs = 255
#c.load('./checkpoints/model-2022-12-08_11:28:47.dict')
for i in range(epochs):
  for batch in tqdm(c.train_loader):
    c.step(batch)
  if i % 15 == 0:  
    c.save()

In [ ]:
c.sample()